In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from backend.preprocessing import preprocess_text
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


dataset = pd.read_csv('data.csv', delimiter=',', header=0, names=['review','sentiment'])

# Metin ön işleme
dataset['cleaned_review'] = dataset['review'].apply(preprocess_text)

# Etiketleri sayısal forma dönüştürme
dataset['label'] = dataset['sentiment'].map({'positive':1, 'negative':0})

# Eğitim/Test ayrımı
X = dataset['cleaned_review'].values
y = dataset['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Bag-of-Words
bow_vectorizer = CountVectorizer(ngram_range=(1, 1)) #ngram
X_train_bow = bow_vectorizer.fit_transform(X_train)
# Öznitelik sayısını öğrenme
print(f"Öznitelik Sayısı (BoW): {X_train_bow.shape[1]}")
X_test_bow = bow_vectorizer.transform(X_test)


# TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1)) #ngram
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# Öznitelik sayısını öğrenme
print(f"Öznitelik Sayısı (TF-IDF): {X_train_tfidf.shape[1]}")
X_test_tfidf = tfidf_vectorizer.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# KNN Modeli
knn_model_bow = KNeighborsClassifier(n_neighbors=5,metric="cosine")
knn_model_bow.fit(X_train_bow, y_train)

# Tahmin yap
y_pred_bow = knn_model_bow.predict(X_test_bow)

# Sonuçları değerlendirme
print("KNN with Bag-of-Words:")
print(classification_report(y_test, y_pred_bow))

# KNN Modeli
knn_model_tfidf = KNeighborsClassifier(n_neighbors=5,metric="cosine")
knn_model_tfidf.fit(X_train_tfidf, y_train)

# Tahmin yap
y_pred_tfidf = knn_model_tfidf.predict(X_test_tfidf)

# Sonuçları değerlendirme
print("KNN with TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Öznitelik Sayısı (BoW): 144577
Öznitelik Sayısı (TF-IDF): 144577
KNN with Bag-of-Words:
              precision    recall  f1-score   support

           0       0.71      0.74      0.73      4961
           1       0.73      0.71      0.72      5039

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000

KNN with TF-IDF:
              precision    recall  f1-score   support

           0       0.81      0.71      0.76      4961
           1       0.74      0.84      0.79      5039

    accuracy                           0.77     10000
   macro avg       0.78      0.77      0.77     10000
weighted avg       0.78      0.77      0.77     10000



In [25]:
import joblib

# Model ve vektörleştiricileri kaydet
joblib.dump(knn_model_tfidf, 'models/knn/knn_model_tfidf.joblib')
joblib.dump(tfidf_vectorizer, 'models/knn/knn_tfidf_vectorizer.joblib')

# Eğer Bag-of-Words modeli de kayıt edilecekse
joblib.dump(knn_model_bow, 'models/knn/knn_model_bow.joblib')
joblib.dump(bow_vectorizer, 'models/knn/knn_bow_vectorizer.joblib')


['models/knn/knn_bow_vectorizer.joblib']

In [40]:
# Kayıtlı model ve vektörleştiricileri yükle
loaded_knn_model = joblib.load('models/knn/knn_model_tfidf.joblib')
loaded_tfidf_vectorizer = joblib.load('models/knn/knn_tfidf_vectorizer.joblib')

# Yeni gelen bir yorum örneği
new_review = "i disslike this movie"

# Ön işleme adımları
processed_new_review = preprocess_text(new_review)  # preprocess_text fonksiyonunu tekrar kullanıyoruz

# Veriyi vektörleştirme
X_new = loaded_tfidf_vectorizer.transform([processed_new_review])

# Tahmin
prediction = loaded_knn_model.predict(X_new)
print("Tahmin Sonucu:", prediction[0]) 


Tahmin Sonucu: 0
